# **Lab 10: Database Design and Build - Case Questions**

## Compute ID: bab3fq

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

## **Case Question 1**
### Which courses are currently included (active) in the program? Include the course mnemonic and course name for each. 

In [ ]:
SELECT
mnemonic,
course_name
FROM COURSES_ERD
WHERE active = 'TRUE';

## Case Question 2
### **Which courses were included in the program, but are no longer active? Include the course mnemonic and course name for each.**

In [ ]:
SELECT
mnemonic,
course_name
FROM courses_erd
WHERE active = 'FALSE';

## Case Question 3
### **Which instructors are not current employees?**

In [ ]:
SELECT
instructor_name
FROM instructors_erd
WHERE active = 'FALSE';

## Case Question 4
### **For each course (active and inactive), how many learning outcomes are there?**

In [ ]:
SELECT
c.course_name,
COUNT(DISTINCT lo.learning_outcome) AS Learning_Outcome_Count
FROM LEARNING_OUTCOMES_ERD lo
LEFT JOIN COURSES_ERD  c
ON lo.course_id = c.course_id
GROUP BY c.course_name
ORDER BY Learning_Outcome_Count DESC;

## Case Question 5
### **Are there any courses with no learning outcomes? If so, provide their mnemonics and names.**

In [ ]:
SELECT
c.mnemonic,
c.course_name
FROM COURSES_ERD c
LEFT JOIN LEARNING_OUTCOMES_ERD lo
ON c.course_id = lo.course_id
WHERE lo.course_id IS NULL;

## Case Question 6
### **Which courses include SQL as a learning outcome? Include the learning outcome descriptions, course mnemonics, and course names in your solution.**

In [ ]:
SELECT
c.mnemonic,
c.course_name,
lo.learning_outcome
FROM LEARNING_OUTCOMES_ERD lo
LEFT JOIN COURSES_ERD  c
ON lo.course_id = c.course_id
WHERE
lo.learning_outcome LIKE '%SQL%';

## Case Question 7
### **Who taught course ds5100 in Summer 2021?**

In [ ]:
SELECT
i.instructor_name
FROM COURSE_ASSIGNMENT_ERD ca
LEFT JOIN instructors_erd i
ON ca.instructor_id = i.instructor_id
LEFT JOIN courses_erd c
ON ca.course_id = c.course_id
WHERE
c.mnemonic = 'ds5100'
AND ca.term = 'summer2021';

## Case Question 8
### **Which instructors taught in Fall 2021? Order their names alphabetically, making sure the names are unique.**

In [ ]:
SELECT DISTINCT
i.instructor_name
FROM COURSE_ASSIGNMENT_ERD ca
LEFT JOIN INSTRUCTORS_ERD i
ON ca.instructor_id = i.instructor_id
WHERE ca.term = 'fall2021'
ORDER BY i.instructor_name;

## Case Question 9
### **How many courses did each instructor teach in each term? Order your results by term and then instructor.**

In [ ]:
SELECT
i.instructor_name,
ca.term,
COUNT(ca.assignment_id) AS course_count
FROM course_assignment_erd ca
LEFT JOIN instructors_erd i
on ca.instructor_id = i.instructor_id
GROUP BY ca.term, i.instructor_name
ORDER BY ca.term, instructor_name;

## Case Question 10a
## **Which courses had more than one instructor for the same term? Provide the mnemonic and term for each. Note this occurs in courses with multiple sections.**

In [ ]:
SELECT
c.mnemonic,
ca.term
FROM course_assignment_erd ca
LEFT JOIN courses_erd c
ON ca.course_id = c.course_id
GROUP BY c.mnemonic, ca.term
HAVING COUNT(DISTINCT ca.instructor_id) > 1
ORDER BY ca.term, c.mnemonic;

## Case Question 10b
### **For courses with multiple sections, provide the term, course mnemonic, and instructor name for each. Hint: You can use your result from 10a in a subquery or WITH clause.**

In [ ]:
WITH courses_multiple_instructors AS (
    SELECT
    c.mnemonic,
    ca.term,
    ca.course_id
    FROM course_assignment_erd ca
    LEFT JOIN courses_erd c
    ON ca.course_id = c.course_id
    GROUP BY c.mnemonic, ca.term, ca.course_id
    HAVING COUNT(DISTINCT ca.instructor_id) > 1
)
SELECT
cmi.term,
cmi.mnemonic,
i.instructor_name
FROM courses_multiple_instructors cmi
LEFT JOIN COURSE_ASSIGNMENT_ERD ca
on cmi.course_id = ca.course_id
AND cmi.term = ca.term
LEFT JOIN INSTRUCTORS_ERD i
ON ca.instructor_id = i.instructor_id
ORDER BY cmi.term, cmi.mnemonic, i.instructor_name;

In [ ]:
test_df = session.sql("""
SELECT 
mnemonic, 
course_name
FROM courses_erd
WHERE active = 'FALSE'
""")
test_df

In [ ]:
import pandas as pd

data = [
    ('CS101', 'Introduction to Computer Science'),
    ('MATH201', 'Calculus I'),
    ('BIO101', 'Biology I')
]

# Create a DataFrame from the sample data
test_df = pd.DataFrame(data, columns=['mnemonic', 'course_name'])

# Show the DataFrame
print(test_df)